///---
layout: post
title: "Linux Terminal"
comments: true
share: true
date: 2020-08-02 13:44:00
description: Linux Terminal 설정 정리
tags: note
toc: false
sitemap :
    changefreq : daily
    priority : 1.0
///---

# Linux Terminal 설정

## Zsh

### Zsh 설치
```sh
sudo apt install zsh
sudo chsh -s /bin/zsh
sudo chsh -s /bin/zsh 아이디
```

재부팅

## Oh-my-zsh

### 설치

```sh
curl -L https://raw.github.com/robbyrussell/oh-my-zsh/master/tools/install.sh | sh
```

### Theme 설정

~/.zshrc에서 테마를 변경한다.

```sh
ZSH_THEME="agnoster"
```

~/.oh-my-zsh/themes/agnoster.zsh-theme에서 prompt의 사용자 명을 지운다.

```sh
prompt_context(){}
```

~/.oh-my-zsh/themes/agnoster.zsh-theme의 ## Main prompt에 다음을 추가하여 multi-line command를 설정한다.
(prompt_newline는 parser문제로 여기에 붙여넣기 불가.)


```sh
# 멀티라인 적용, 커버모양 변경

## Main prompt
build_prompt() {
  RETVAL=$?
  prompt_status
  prompt_virtualenv
  prompt_context
  prompt_dir
  prompt_git
  prompt_bzr
  prompt_hg
  prompt_newline # 멀티라인 적용
  prompt_end
}
```

### Plugin 설치
```sh
# zsh-syntax-highlighting
git clone https://github.com/zsh-users/zsh-syntax-highlighting.git \${ZSH_CUSTOM:-~/.oh-my-zsh/custom}/plugins/zsh-syntax-highlighting

# zsh-autosuggestions
git clone git://github.com/zsh-users/zsh-autosuggestions \$ZSH_CUSTOM/plugins/zsh-autosuggestions

# zsh-vim
git clone https://github.com/softmoth/zsh-vim-mode.git \${ZSH_CUSTOM:-~/.oh-my-zsh/custom}/plugins/zsh-vim-mode
```

```sh
plugins=(
  git
  zsh-syntax-highlighting
  zsh-autosuggestions
  zsh-vim-mode
)
```

## Tmux

### 설치

```sh
sudo apt install tmux
```

### vim 모드 설정

~/.tmux.conf에 저장한다. (ref [.tmux.conf](https://gist.github.com/tsl0922/d79fc1f8097dde660b34#file-tmux-conf))

```sh
# vim style tmux config

# use C-a, since it's on the home row and easier to hit than C-b
set-option -g prefix C-a
unbind-key C-a
bind-key C-a send-prefix
set -g base-index 1

# Easy config reload
bind-key R source-file ~/.tmux.conf \; display-message "tmux.conf reloaded."

# vi is good
setw -g mode-keys vi

# mouse behavior
setw -g mouse on

set-option -g default-terminal screen-256color

bind-key : command-prompt
bind-key r refresh-client
bind-key L clear-history

bind-key space next-window
bind-key bspace previous-window
bind-key enter next-layout

# use vim-like keys for splits and windows
bind-key v split-window -h
bind-key s split-window -v
bind-key h select-pane -L
bind-key j select-pane -D
bind-key k select-pane -U
bind-key l select-pane -R

# smart pane switching with awareness of vim splits
bind -n C-h run "(tmux display-message -p '#{pane_current_command}' | grep -iqE '(^|\/)vim$' && tmux send-keys C-h) || tmux select-pane -L"
bind -n C-j run "(tmux display-message -p '#{pane_current_command}' | grep -iqE '(^|\/)vim$' && tmux send-keys C-j) || tmux select-pane -D"
bind -n C-k run "(tmux display-message -p '#{pane_current_command}' | grep -iqE '(^|\/)vim$' && tmux send-keys C-k) || tmux select-pane -U"
bind -n C-l run "(tmux display-message -p '#{pane_current_command}' | grep -iqE '(^|\/)vim$' && tmux send-keys C-l) || tmux select-pane -R"
bind -n 'C-\' run "(tmux display-message -p '#{pane_current_command}' | grep -iqE '(^|\/)vim$' && tmux send-keys 'C-\\') || tmux select-pane -l"
bind C-l send-keys 'C-l'

bind-key C-o rotate-window

bind-key + select-layout main-horizontal
bind-key = select-layout main-vertical

set-window-option -g other-pane-height 25
set-window-option -g other-pane-width 80
set-window-option -g display-panes-time 1500
set-window-option -g window-status-current-style fg=magenta

bind-key a last-pane
bind-key q display-panes
bind-key c new-window
bind-key t next-window
bind-key T previous-window

bind-key [ copy-mode
bind-key ] paste-buffer

# Setup 'v' to begin selection as in Vim
bind-key -T copy-mode-vi v send -X begin-selection
bind-key -T copy-mode-vi y send -X copy-pipe-and-cancel "reattach-to-user-namespace pbcopy"

# Update default binding of `Enter` to also use copy-pipe
unbind -T copy-mode-vi Enter
bind-key -T copy-mode-vi Enter send -X copy-pipe-and-cancel "reattach-to-user-namespace pbcopy"

# Status Bar
set-option -g status-interval 1
set-option -g status-style bg=black
set-option -g status-style fg=white
set -g status-left '#[fg=green]#H #[default]'
set -g status-right '%a%l:%M:%S %p#[default] #[fg=blue]%Y-%m-%d'

set-option -g pane-active-border-style fg=yellow
set-option -g pane-border-style fg=cyan

# Set window notifications
setw -g monitor-activity on
set -g visual-activity on

# Enable native Mac OS X copy/paste
set-option -g default-command "/bin/bash -c 'which reattach-to-user-namespace >/dev/null && exec reattach-to-user-namespace $SHELL -l || exec $SHELL -l'"

# Allow the arrow key to be used immediately after changing windows
set-option -g repeat-time 0
```

### zsh 실행 시 tmux 자동 연결

~/.zshrc plugins 밑에 다음을 추가한다.

```sh
if command -v tmux &> /dev/null && [ -n "$PS1" ] && [[ ! "$TERM" =~ screen ]] && [[ ! "$TERM" =~ tmux ]] && [ -z "$TMUX" ]; then
  exec tmux
fi
```